In [1]:
%matplotlib inline

from itertools import islice

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import keras
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import EarlyStopping, LearningRateScheduler, \
                            ReduceLROnPlateau, TensorBoard
import keras.backend as K

Using TensorFlow backend.


In [2]:
def scaler(X, max_num=100):
    return X.astype(np.float) / max_num


def data_generator(batch_size=128, max_num=100, excludes=[50], num_class=201, scaler=None,
                   is_train=True):
    while True:
        if is_train:
            X = np.random.randint(low=0, high=max_num+1, size=(batch_size, 2)).astype(np.float)
            rand_iter = iter(lambda: np.random.randint(max_num+1), max_num+1)
            X[np.isin(X, excludes)] = next(x for x in rand_iter if x not in excludes)
        else:
            X1 = np.random.choice(excludes, size=batch_size)
            X2 = np.random.randint(low=0, high=max_num+1, size=batch_size)
            X = np.vstack((X1, X2)).T
        y = np_utils.to_categorical(X.sum(axis=1), num_class)
        if scaler:
            X = scaler(X)
        yield X, y

In [3]:
# experiment settings
max_num = 100
val_num = 1
excludes = np.random.choice(max_num, size=val_num, replace=False)

# model parameters
num_class = 2*max_num + 1
n_hidden_1 = 50
n_hidden_2 = 100

# training parameters
num_epoch = 500
batch_size = 1024
decay = 0.1
learning_rate = 0.1

# network architecture
input_layer = Input(shape=(2,))
x = Dense(n_hidden_1, activation='relu', name='Dense_1')(input_layer)
x = Dense(n_hidden_2, activation='relu', name='Dense_2')(x)
output_layer = Dense(num_class, activation='softmax', name='Output')(x)

# model creation
model = Model(input_layer, output_layer)
print model.summary()

# training model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
K.set_value(model.optimizer.lr, learning_rate)
train_gen = data_generator(batch_size=batch_size,
                           max_num=max_num,
                           excludes=excludes,
                           num_class=num_class,
                           scaler=scaler)
valid_gen = data_generator(batch_size=batch_size,
                           max_num=max_num,
                           excludes=excludes,
                           num_class=num_class,
                           scaler=scaler,
                           is_train=False)
callbacks = [EarlyStopping(monitor='loss', min_delta=0.001, patience=3),
             LearningRateScheduler(lambda epoch:learning_rate/(1 + decay * epoch)),
             ReduceLROnPlateau(monitor='loss'),
             TensorBoard()]
history = model.fit_generator(train_gen,
                              steps_per_epoch=100,
                              epochs=num_epoch,
                              callbacks=callbacks,
                              validation_data=valid_gen,
                              validation_steps=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
Dense_1 (Dense)              (None, 50)                150       
_________________________________________________________________
Dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
Output (Dense)               (None, 201)               20301     
Total params: 25,551
Trainable params: 25,551
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
100/100 [==============================] - 0s - loss: 3.6482 - acc: 0.0652 - val_loss: 3.0981 - val_acc: 0.1270
Epoch 2/500
100/100 [==============================] - 0s - loss: 2.6573 - acc: 0.1440 - val_loss: 2.5415 - val_acc: 0.1416
Epoch 3/500
100/1

100/100 [==============================] - 0s - loss: 0.4029 - acc: 0.9187 - val_loss: 0.4249 - val_acc: 0.8936
Epoch 60/500
100/100 [==============================] - 0s - loss: 0.3943 - acc: 0.9213 - val_loss: 0.4317 - val_acc: 0.9072
Epoch 61/500
100/100 [==============================] - 0s - loss: 0.3925 - acc: 0.9243 - val_loss: 0.5603 - val_acc: 0.8906
Epoch 62/500
100/100 [==============================] - 0s - loss: 0.3863 - acc: 0.9251 - val_loss: 0.5976 - val_acc: 0.9072
Epoch 63/500
100/100 [==============================] - 0s - loss: 0.3709 - acc: 0.9283 - val_loss: 0.4403 - val_acc: 0.9043
Epoch 64/500
100/100 [==============================] - 0s - loss: 0.3780 - acc: 0.9219 - val_loss: 0.4713 - val_acc: 0.9365
Epoch 65/500
100/100 [==============================] - 0s - loss: 0.3702 - acc: 0.9237 - val_loss: 0.4662 - val_acc: 0.9307
Epoch 66/500
100/100 [==============================] - 0s - loss: 0.3588 - acc: 0.9334 - val_loss: 0.4044 - val_acc: 0.9443
Epoch 67/500
